# Getting Started with OPERA DIST Product
---

**This notebook serves as an introduction to the OPERA Land Surface Disturbance (DIST) product and visualizing relevant raster layers for wildfire applications.**

<font color='red'> Download the provisional data at https://www.jpl.nasa.gov/go/opera/products/dist-product-suite </font>

In [ ]:
# Notebook dependencies
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

import xarray as xr
import numpy as np
import rioxarray as rio

import hvplot.xarray
from matplotlib.colors import ListedColormap
from bokeh.models import FixedTicker
import geoviews as gv
gv.extension('bokeh')



### Product Background
---

The DIST product maps per pixel vegetation disturbance (specifically, vegetation cover loss) from the Harmonized Landsat-8 and Sentinel-2 A/B (HLS) scenes. Vegetation disturbance is mapped when there is an indicated decrease in vegetation cover within an HLS pixel. This notebook focuses on relevant layers within the **DIST_ALERT** product for wildfire applications, which is released at the cadence of HLS imagery. 



### Metadata
---

HLS products provide surface reflectance (SR) data from the Operational Land Imager (OLI) aboard the Landsat-8 remote sensing satellite and the Multi-Spectral Instrument (MSI) aboard the Sentinel-2 A/B remote sensing satellite. HLS products are distributed over projected map coordinates aligned with the Military Grid Reference System (MGRS). Each tile covers 109.8 kilometers squared divided into 3660 rows and 3660 columns at 30 meter pixel spacing. Each tile overlaps neighbors by 4900 meters in each direction.



### Raster Layers
___

The **DIST_ALERT** product is distributed as a set of Cloud-Optimized GeoTIFF (COG) files to enable download of only particular layers of interest to a given user. All L3 DIST layers are stored in files following GeoTIFF format specifications.

___

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DIST-ALERT-HLS_T10TEM_20220815T185931Z_20220817T153514Z_S2A_30_v0.1_VEG-ANOM-MAX.tif'

In [ ]:
data = rio.open_rasterio(LOCAL_PATH)
data

In [ ]:
crs = data.rio.crs
crs

In [ ]:
dist = data.rename({'x':'longitude', 'y':'latitude', 'band':'band'}).squeeze()
dist

In [ ]:
# Creates basemap
base = gv.tile_sources.EsriTerrain.opts(width=1000, height=1000, padding=0.1)

## **Band 1: Maximum Vegetation Anomaly Value (VEG_ANOM_MAX)**
***

**Data Type:** UInt8<br>
**Description:** Difference between historical and current year observed vegetation cover at the date of maximum decrease, measured on scale from 0-100%<br>

In [ ]:
base = gv.tile_sources.EsriNatGeo.opts(width=1000, height=1000, padding=0.1)
veg_anom_max = dist
veg_anom_max.hvplot.image(
                          x='longitude', 
                          y='latitude', 
                          crs=crs, 
                          rasterize=True, 
                          dynamic=True, 
                          aspect='equal', 
                          frame_width=500, 
                          frame_height=500, 
                          cmap='hot_r', 
                          clim=(0,100), alpha=0.8).opts(title=f"VEG_ANOM_MAX", xlabel='Longitude', ylabel='Latitude') * base

## **Band 2: Date of Initial Vegetation Disturbance (VEG_DIST_DATE)**
***

**Data Type:** Int16<br>
**Description:** Day of first loss anomaly detection in the last year, denoted as the number of days since December 31st, 2020.<br>

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DIST-ALERT-HLS_T10TEM_20220815T185931Z_20220817T153514Z_S2A_30_v0.1_VEG-DIST-DATE.tif'

In [ ]:
data = rio.open_rasterio(LOCAL_PATH)
crs = data.rio.crs
veg_dist_date = data.rename({'x':'longitude', 'y':'latitude', 'band':'band'}).squeeze()

In [ ]:
veg_dist_date.hvplot.image(x='longitude', 
                           y='latitude', 
                           crs=crs, 
                           rasterize=True, 
                           dynamic=True, 
                           aspect='equal', 
                           frame_width=500, 
                           frame_height=500, 
                           cmap='hot_r', 
                           alpha=0.8).opts(title=f"VEG_DIST_DATE", xlabel='Longitude', ylabel='Latitude',clim=(0, 592)) * base


## **Band 3: Vegetation Disturbance Status (VEG_DIST_STATUS)**
***

**Data Type:** UInt8<br>
**Description:** Indication of vegetation cover loss (vegetation disturbance); "provisional" is used from the first detection until vegetation disturbance is detected for consecutive number of HLS scenes, when it is then labeled "confirmed."<br>

In [ ]:
LOCAL_PATH = Path('..') / 'assets' / 'OPERA_L3_DIST-ALERT-HLS_T10TEM_20220815T185931Z_20220817T153514Z_S2A_30_v0.1_VEG-DIST-STATUS.tif'

In [ ]:
data = rio.open_rasterio(LOCAL_PATH)
crs = data.rio.crs
veg_dist_status = data.rename({'x':'longitude', 'y':'latitude', 'band':'band'}).squeeze()

In [ ]:
veg_dist_status.hvplot.image(x='longitude', 
                             y='latitude', 
                             crs=crs, 
                             rasterize=True, 
                             dynamic=True, 
                             aspect='equal', 
                             frame_width=500, 
                             frame_height=500, 
                             cmap='hot_r', 
                             alpha=0.8).opts(title=f"VEG_DIST_STATUS", clim=(0,4), colorbar_opts={'ticker': FixedTicker(ticks=[0, 1, 2, 3, 4])}, xlabel='Longitude', ylabel='Latitude') * base


**Layer Values:**<br> 
* **0:** No disturbance<br>
* **1:** Provisional (**first detection**) Disturbance with vegetation cover change <50% <br>
* **2:** Confirmed (**recurrent detection**) Disturbance with vegetation cover change < 50% <br> 
* **3:** Provisional Disturbance with vegetation cover change ≥ 50% <br>
* **4:** Confirmed Disturbance with vegetation cover change ≥ 50%  <br> 